Batch-8 Detection and localisation of myocardial infraction ---- Part2 (We have preprocessed the data , extracted the features and saved the trainingset and testing set )

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.11.0


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import itertools
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
%matplotlib inline
     

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Here we Imported the  trainedset and trainset we created in feature extraction

In [5]:
X_train = np.load('/content/drive/My Drive/majorproject/XTrain.npy')

X_test = np.load('/content/drive/My Drive/majorproject/XTest.npy')

y_train = np.load('/content/drive/My Drive/majorproject/yTrain.npy')
  
y_test = np.load('/content/drive/My Drive/majorproject/yTest.npy')

In [6]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, Activation
from tensorflow.keras.layers import GlobalAveragePooling1D, BatchNormalization, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.regularizers import l1, l2, l1_l2 

In [7]:
    reg2 = l2(0.01)  
    model = Sequential()
    model.add(Conv1D(filters=128,kernel_size=1, activation="relu", input_shape = (36, 1)))
    model.add(Conv1D(filters=128,kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=128, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=128, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size = 1))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(units=256,activation="relu", kernel_regularizer = reg2, bias_regularizer = reg2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(units=256,activation="relu", kernel_regularizer = reg2, bias_regularizer = reg2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(units=10, activation="softmax"))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 36, 128)           256       
                                                                 
 conv1d_1 (Conv1D)           (None, 36, 128)           16512     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 36, 128)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 36, 128)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 36, 128)           16512     
                                                                 
 conv1d_3 (Conv1D)           (None, 36, 128)           16512     
                                                        

In [9]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x = X_train, y = y_train, batch_size = 128, epochs = 70)

Epoch 1/70
221/221 [==============================] - 36s 45ms/step - loss: 7.5579 - accuracy: 0.2842
Epoch 2/70
221/221 [==============================] - 9s 40ms/step - loss: 5.0108 - accuracy: 0.4570
Epoch 3/70
221/221 [==============================] - 9s 40ms/step - loss: 3.6945 - accuracy: 0.5548
Epoch 4/70
221/221 [==============================] - 9s 41ms/step - loss: 2.9082 - accuracy: 0.6133
Epoch 5/70
221/221 [==============================] - 9s 41ms/step - loss: 2.3947 - accuracy: 0.6532
Epoch 6/70
221/221 [==============================] - 9s 42ms/step - loss: 2.0497 - accuracy: 0.6791
Epoch 7/70
221/221 [==============================] - 9s 43ms/step - loss: 1.7946 - accuracy: 0.7088
Epoch 8/70
221/221 [==============================] - 9s 42ms/step - loss: 1.6037 - accuracy: 0.7269
Epoch 9/70
221/221 [==============================] - 9s 42ms/step - loss: 1.4460 - accuracy: 0.7489
Epoch 10/70
221/221 [==============================] - 9s 42ms/step - loss: 1.3217 - accur

In [10]:
score = model.evaluate(X_test, y_test, batch_size = 128)

74/74 [==============================] - 4s 21ms/step - loss: 0.3588 - accuracy: 0.9518


In [ ]:
model.save('/content/drive/My Drive/majorproject/lstm_model.h5') 

ACCURACY ACHIEVED BY MODEL

In [11]:
print("Accuracy achieved ", round(score[1]*100, 2))

Accuracy achieved  95.18


In [12]:
y_pred = np.argmax(model.predict(X_test), axis = -1)

295/295 [==============================] - 6s 9ms/step


In [14]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred)    

a = matrix.diagonal()/matrix.sum(axis = 1) 

In [15]:
for i in range(len(a)):
  if(i==0):
    temp = 'ALMI'
  elif(i==1):
    temp = 'AMI'
  elif(i==2):
    temp = 'ASMI'
  elif(i==3):
    temp = 'ILMI'
  elif(i==4):
    temp = 'IMI'
  elif(i==5):
    temp = 'IPLMI'
  elif(i==6):
    temp = 'IPMI'
  elif(i==7):
    temp = 'LMI'
  elif(i==8):
    temp = 'NORM'
  else:
    temp = 'PMI'
  print('Accuracy of the model on   ' + temp + ' = ', np.round(a[i]*100, 2))

Accuracy of the model on   ALMI =  99.91
Accuracy of the model on   AMI =  97.63
Accuracy of the model on   ASMI =  85.57
Accuracy of the model on   ILMI =  96.35
Accuracy of the model on   IMI =  82.66
Accuracy of the model on   IPLMI =  100.0
Accuracy of the model on   IPMI =  99.87
Accuracy of the model on   LMI =  100.0
Accuracy of the model on   NORM =  85.82
Accuracy of the model on   PMI =  100.0


In [17]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1128
           1       0.94      0.98      0.96       928
           2       0.85      0.86      0.85       797
           3       0.95      0.96      0.96       987
           4       0.89      0.83      0.86       969
           5       1.00      1.00      1.00       911
           6       1.00      1.00      1.00       745
           7       0.98      1.00      0.99      1112
           8       0.89      0.86      0.87       783
           9       1.00      1.00      1.00      1061

    accuracy                           0.95      9421
   macro avg       0.95      0.95      0.95      9421
weighted avg       0.95      0.95      0.95      9421



In [18]:
from sklearn.metrics import confusion_matrix
def multiclass_sensitivity(true_labels, predicted_labels, num_classes):
    cm = confusion_matrix(true_labels, predicted_labels)
    sensitivities = []
    for i in range(num_classes):
        tp = cm[i][i]
        fn = sum(cm[i, :]) - tp
        sensitivity = tp / (tp + fn)
        sensitivities.append(sensitivity)
    avg_sensitivity = sum(sensitivities) / num_classes
    return avg_sensitivity
  

In [19]:
multiclass_sensitivity(y_test,y_pred,10)

0.9478161975908795

In [20]:
from sklearn.metrics import confusion_matrix
def multiclass_specificity(true_labels, predicted_labels, num_classes):
    cm = confusion_matrix(true_labels, predicted_labels)
    specificities = []
    for i in range(num_classes):
        tn = (cm.sum() - cm[i].sum() - cm[:, i].sum() + cm[i, i])
        fp = cm[:, i].sum() - cm[i, i]
        specificity = tn / (tn + fp)
        specificities.append(specificity)
    avg_specificity = sum(specificities) / num_classes
    return avg_specificity

In [21]:
multiclass_specificity(y_test,y_pred,10)

0.9946711753146292